In [18]:
import numpy as np
import cv2

In [27]:
imagePoints = np.array([
    (853, 644),
    (252, 435),
    (768, 452),
    (563, 651)
], dtype=np.float32)
worldPoints = np.array([
    [0,1,0],
    [0,-1,0],
    [-0.6294,0,0],
    [0.5986,0.1945,0]
], dtype=np.float32)

WIDTH = 1280
HEIGHT = 720

focal_length = 2.1
#focal_length = 3.6651 # calculated from sensor width and fov
principal_point = (WIDTH//2, HEIGHT//2)

camera_matrix = np.array([
    [focal_length/0.003, 0, 640],
        [0, focal_length/0.003, 360],
        [0, 0, 1]
    ], dtype=np.float32)

dist_coeffs = np.array([1.40017137e-14, 2.14617695e-10, -1.38004061e-16, 2.35596742e-16, -6.64757209e-15], dtype=np.float32)


In [28]:
success, rotation_vector, translation_vector, _ = cv2.solvePnPRansac(worldPoints, imagePoints, camera_matrix, dist_coeffs)#, flags=cv2.SOLVEPNP_SQPNP)
# PnP rechnet die die Transformation von Weltkoordinaten in Kamerakoordinaten aus
print("Rotation Vector:\n {0}".format(rotation_vector))
print("Translation Vector:\n {0}".format(translation_vector))

rotM = cv2.Rodrigues(rotation_vector)[0]
print("Rotation Matrix:\n {0}".format(rotM))

projMatrix = np.hstack((rotM, translation_vector))
projMatrix = np.vstack((projMatrix, np.array([0,0,0,1])))
camera_pose = np.linalg.inv(projMatrix)
print("Camera Pose:\n {0}".format(camera_pose))

#homogenous_camera_matrix = np.vstack((camera_matrix, np.array([0,0,0], dtype=np.float32)))
homogenous_camera_matrix = np.hstack((camera_matrix, np.array([[0],[0],[0]], dtype=np.float32)))
world2image = np.dot(homogenous_camera_matrix, projMatrix)
worldPoints2= np.hstack((worldPoints, np.array([[1],[1],[1],[1]], dtype=np.float32)))
imagePoints2 = world2image @ worldPoints2.T
for x, y, z in imagePoints2.T:
    x = x / z
    y = y / z
    print(x, y)
print("Original Image Points:\n {0}".format(imagePoints))

Rotation Vector:
 [[-1.19502314]
 [-2.33470515]
 [-0.97997799]]
Translation Vector:
 [[-0.06891478]
 [ 0.57787268]
 [ 2.0068106 ]]
Rotation Matrix:
 [[-0.58837329  0.80851857  0.01070471]
 [ 0.57393726  0.40826571  0.7098698 ]
 [ 0.56957255  0.42381226 -0.70425158]]
Camera Pose:
 [[-0.58837329  0.57393726  0.56957255 -1.5152345 ]
 [ 0.80851857  0.40826571  0.42381226 -1.03071766]
 [ 0.01070471  0.7098698  -0.70425158  1.00382288]
 [ 0.          0.          0.          1.        ]]
852.9999935627342 643.9999914169267
252.00000643735962 435.0000007450269
767.9999986289616 451.9999986887364
563.9973642783498 648.2852584530273
Original Image Points:
 [[853. 644.]
 [252. 435.]
 [768. 452.]
 [563. 651.]]
